# Basketball-Reference Scraper Overview
The following code will walk you through how to scrape NBA player and game data from www.basketball-reference.com and input it into a Pandas database. In other scripts within this directory, we will be using the data captured here to run analyses that will help the average user ask both broad and specific questions related to the NBA. We will probe on what statistics and criteria are important for an NBA team to win an NBA championship, how the league has evolved year-over-year, touch on the GOAT debate, and ultimately, build algorithms that can (hopefully) help us all beat Vegas lines consistently so that we can all retire from our day jobs and gamble on the NBA for the rest of our careers. 

None of this could have been done without the tireless and comprehensive effort of those who work at [Basketball Reference](http://www.basketball-reference.com) providing an open-source, API-friendly database containing millions of datapoints from which the entirety of this codebase is built. 

For any questions/concerns, feel free to reach out to me directly at rahim.hashim@columbia.edu. And in the case that this is useful to anyone for future projects, please give credit where credit is due, both to [Basketball Reference](http://www.basketball-reference.com) and myself. Enjoy!

***
## The Basics
__Jupyter Notebook__: All of the following code is hosted in a Python 3 Jupyter Notebook. It is recommended to use Anaconda to access the Notebook in order to have synchronously have access to all Python Libraries used in the rest of the code. 

In order to execute and compile code in the notebook, go to the desired code box and press _Shift_ + _Enter_ at the same time. All code below is recommended to be executed from top to bottom in order.

__Python Libraries__: Python is a beautiful language for a number of reasons, one of which is it's immense
amount of pre-built libraries that do much of the heavy lifting in any web-scraping /
data analysis project. When getting familiar with Python and starting a new project, be
sure to look through the internet for a Python library that may help. A comprehensive list
that I often refer to before starting a project is here: [https://github.com/vinta/awesome-python](https://github.com/vinta/awesome-python)

__Installing Libraries__: In case you receive an error upon trying to execute the following box, such as _ModuleNotFoundError: No module named 'numpy'_, go back to your terminal and open a new tab, and install the library using pip: _pip install numpy_

In [1]:
%reload_ext autoreload
import os
import sys
import numpy as np
import pandas as pd
from pprint import pprint
import matplotlib.pyplot as plt
from collections import Counter, OrderedDict, defaultdict
pd.options.mode.chained_assignment = None  # default='warn'

ROOT = '/content/drive/MyDrive/Projects/nba-prediction-algorithm/NBA-Prediction-Algorithms/' #@param ['/content/drive/MyDrive/Projects/nba-prediction-algorithm/NBA-Prediction-Algorithms/']  

# add (non-Python) helper functions
def add_helpers():
  '''
  add_helper mounts google drive and adds
  helper functions to the sys.path
  '''

  # if running on juypter/google colab, mount to google drive
  if 'google.colab' in str(get_ipython()): 
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    os.chdir(ROOT)

  helper_dir_path = ROOT + 'helper/'
  print('\nHelpers:')
  pprint(sorted(os.listdir(helper_dir_path)))
  sys.path.append(helper_dir_path) # set to path of notebook

add_helpers()

Mounted at /content/drive

Helpers:
['Regions.py',
 'TeamNames.py',
 '__pycache__',
 'bettingLinesScraper.py',
 'fuzzy_lookup.py',
 'game_log_scraper.py',
 'gamelog_scraper_openai.ipynb',
 'gamelogscraper.ipynb',
 'meta_info_scraper.py',
 'player_info_scraper.py',
 'player_scraper.py',
 'player_table_scraper.py',
 'recordDateScraper.py',
 'teamsScraper.py']


***
## Creating Databases
Pandas databases are a powerful tool to query large amounts of data, as we will be doing here. For that reason, we are going to insert all of the data scraped above into a Pandas database. The below code will take player overview data from playerHash and insert it into player_df<br>
>For documentation on pandas: https://pypi.org/project/pandas/

***
## Scraping Player Data
### Biometrics and season + career statistics

playerScraper and metaDataScraper will be doing most of the work to scrape data on each player's background and physical attributes.<br>
> Example Overview Source (last name starting with a): https://www.basketball-reference.com/players/a/<br>
> Example meta-data (Karim Abdul-Jabbar): https://www.basketball-reference.com/players/a/abdulka01.html<br>
> For documentation on requests(): https://realpython.com/python-requests/<br>
> For documentation on BeautifulSoup: https://www.crummy.com/software/BeautifulSoup/bs4/doc/

___Time Estimates:___ This is the most computationally-intensive function in the program, requiring many url requests in order to complete.<br>
>*Without threading:* ~1hr<br>
>*With threading:* ~15min<br>

In [2]:
from player_scraper import scrape_all_players

list_players_meta, list_players_data, list_players_gamelogs = scrape_all_players(ROOT, THREAD_FLAG=True)

Running meta_info_scraper.py
  Start Time: 20:39:00.25
  Threading activated...
	  x' Players Captured:  0
	  q' Players Captured:  6
	  z' Players Captured:  20
	  u' Players Captured:  11
https://www.basketball-reference.com/players/n/napolpa01/gamelog/1949 missing, check gamelog


Exception in thread Thread-25:
Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/content/drive/MyDrive/Projects/nba-prediction-algorithm/NBA-Prediction-Algorithms/helper/meta_info_scraper.py", line 83, in meta_info_scraper
    df_meta, df_player_data, df_gamelogs = player_info_scraper(player_name, playerURL)
  File "/content/drive/MyDrive/Projects/nba-prediction-algorithm/NBA-Prediction-Algorithms/helper/player_info_scraper.py", line 75, in player_info_scraper
    df_player, df_gamelogs = player_table_scraper(playerName, soup)
  File "/content/drive/MyDrive/Projects/nba-prediction-algorithm/NBA-Prediction-Algorithms/helper/player_table_scraper.py", line 120, in player_table_scraper
    gamelogs_df = pd.concat([gamelogs_df,gamelog_df])
  File "/usr/local/lib/python3.7/dist-packages/pandas/core/reshape/co

	  y' Players Captured:  19
https://www.basketball-reference.com/players/h/hansro01/gamelog/1955 missing, check gamelog


Exception in thread Thread-19:
Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/content/drive/MyDrive/Projects/nba-prediction-algorithm/NBA-Prediction-Algorithms/helper/meta_info_scraper.py", line 83, in meta_info_scraper
    df_meta, df_player_data, df_gamelogs = player_info_scraper(player_name, playerURL)
  File "/content/drive/MyDrive/Projects/nba-prediction-algorithm/NBA-Prediction-Algorithms/helper/player_info_scraper.py", line 75, in player_info_scraper
    df_player, df_gamelogs = player_table_scraper(playerName, soup)
  File "/content/drive/MyDrive/Projects/nba-prediction-algorithm/NBA-Prediction-Algorithms/helper/player_table_scraper.py", line 120, in player_table_scraper
    gamelogs_df = pd.concat([gamelogs_df,gamelog_df])
  File "/usr/local/lib/python3.7/dist-packages/pandas/core/reshape/co

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-918152fd0d9f>", line 4, in <module>
    df_players_meta, df_players_data, df_players_gamelogs = scrape_all_players(ROOT, THREAD_FLAG=True)
  File "/content/drive/MyDrive/Projects/nba-prediction-algorithm/NBA-Prediction-Algorithms/helper/player_scraper.py", line 106, in scrape_all_players
    thread.join() # makes sure that the main program waits until all threads have terminated
  File "/usr/lib/python3.7/threading.py", line 1044, in join
    self._wait_for_tstate_lock()
  File "/usr/lib/python3.7/threading.py", line 1060, in _wait_for_tstate_lock
    elif lock.acquire(block, timeout):
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/inter

KeyboardInterrupt: ignored

***
## Concatenating Player Data

Now that all the individual players' bio data, season-wide stats, and gamelogs have  been captured, they can be concatenated to generate 3 DataFrames holding all the players. 

In [ ]:
from player_scraper import concatenate_dfs
df_players_meta, df_players_data, df_players_gamelogs = concatenate_dfs(ROOT, list_players_meta, list_players_data, list_players_gamelogs)

***
## Example Queries (Simple)

The following are example queries we can make across all of the generated tables. As can be seen below, the structure of the DataFrame allows for immense flexibility and speed gains as compared to looking at the website itself. We will utilize this structure for more specific trend-, team-, and era- related investigations.

In [ ]:
df_players_meta.head(0)

In [ ]:
# Player Meta Query
df_large = df_players_meta.loc[(df_players_meta['height']>80) & 
                   (df_players_meta['weight']>30)]

df_large = df_large.dropna(how='all', axis='columns') # drops all columns that are empty
display(df_large[df_large['weight'] == df_large['weight'].max()])

In [ ]:
df_players_data.head(5)

In [ ]:
# Player Data Query
'''
df_career = df_players_data.loc[(df_players_data['season']=='Career') & 
                   (df_players_data['season_playoffs']=='season') &
                   (df_players_data['data_type']=='advanced')]
'''
df_players_data['mp_clean'] = df_players_data['mp_per_g'].apply(float)
#df_career[df_career['ws_per_48'].notna()].sort_values('ws_per_48', ascending=False)
df_players_data[(df_players_data['season'] == '2020-21') & 
                (df_players_data['data_type'] == 'per_game') &
                (df_players_data['mp_clean'] > 40)]

In [ ]:
df_players_gamelogs.head(0)

In [ ]:
df_players_gamelogs.head(5)

***
## Single Player Queries

If you don't need to scrape the entire basketball-reference website but instead just want information for one player, you can use the following command.

In [ ]:
from player_scraper import single_player_scraper

pd.set_option('display.max_rows', None)
single_player_info, single_player_df, single_player_gl = single_player_scraper()
single_player_df.head(5)

***
## Scraping Game Data
### Game-logs and team statistics

In [ ]:
single_player_gl

***
## Data Organization
To help us understand how all the data is organized, here's a visual:

In [ ]:
df_career = df_players_data.loc[(df_players_data['season']=='Career') & 
                   (df_players_data['season_playoffs']=='season') &
                   (df_players_data['data_type']=='advanced')]

df_career.dropna(axis='columns')

***
## Meta-Data Analysis
Now that we've scraped all the meta-info on each player, we can start running analyses.

Below, a few simple analyses are included to help you get started. The first set of graphs examine height distribution (left), weight distribution (middle), and shooting handedness (right).

In [ ]:
from Regions import stateDict #stateDict is a Dictionary to help with geography-based analyses
def metaPlot():
  
  height_list = df_players_meta['height'].tolist()
  weight_list = df_players_meta['weight'].tolist()
  #rightCount = 0; leftCount = 0; noHandCount = 0

  #Plot Height Distribution (1, Left)
  f, ax = plt.subplots(1,2)
  #Sets default plot size
  plt.rcParams['figure.figsize'] = (10,8)
  n1, bins1, patches1 = ax[0].hist(height_list, bins=20, density=True, histtype='bar', ec='black')
  #Converting y-axis labels from decimals to percents
  y_vals = ax[0].get_yticks(); ax[0].set_yticklabels(['{:3.1f}%'.format(y*100) for y in y_vals])
  #Converting x-axis labels from inches back to feet
  xticks1 = ['5-0', '5-6', '6-0', '6-6', '7-0', '7-6', '8-0']
  ax[0].set_xticks([60, 66, 72, 78, 84, 90, 96])
  ax[0].set_xticklabels(xticks1)
  ax[0].set_xlim([56,100])
  ax[0].set_xlabel('Height', fontweight='bold', labelpad=10)
  ax[0].set_ylabel('Percent of Players', fontweight='bold', labelpad=10)

  #Plot Weight Distribution (1, Middle)
  ax[1].hist(weight_list, bins='auto', density=True, histtype='bar', ec='black')
  y_vals = ax[1].get_yticks()
  ax[1].set_yticklabels(['{:3.1f}%'.format(y*100) for y in y_vals])
  xticks2 = ['150', '180', '210', '240', '270', '300', '330']
  ax[1].set_xticks([150, 180, 210, 240, 270, 300, 330])
  ax[1].set_xticklabels(xticks2)
  ax[1].set_xlim([120,360])
  ax[1].set_xlabel('Weight', fontweight='bold', labelpad=10)
  ax[1].set_ylabel('Percent of Players', fontweight='bold', labelpad=10)
  
  plt.tight_layout(pad=0.05, w_pad=4, h_pad=1.0)
  f.set_size_inches(18.5, 10.5, forward=True)
  plt.show()
        
metaPlot()

In [ ]:
def geographyPlot():
  countryList = df_players_meta['birthCountry'].tolist()
  #countryList contains all players born in ex-US
  countryList = filter(lambda x: x != 'United States of America', countryList)
  countryList = filter(lambda x: x != '', countryList)
  countryHash = dict(Counter(countryList))
  countryHash = OrderedDict(sorted(countryHash.items(), reverse=True, key=lambda t: t[1]))

  pprint(countryHash.keys())

  #Plot Birth Countries of non-US-Born Players (3)
  f, ax = plt.subplots(1)
  countryList = countryHash.keys(); countryVals = countryHash.values()
  ax.bar(np.arange(len(countryList)), countryVals, ec='black')
  ax.set_xticks(np.arange(len(countryList)))
  ax.set_xticklabels(countryList, rotation=90, ha='right', fontsize=7)
  ax.set_xlabel('Country of Birth', fontweight='bold', labelpad=10)
  ax.set_ylabel('Number of Players', fontweight='bold', labelpad=10)
  
  f.set_size_inches(18.5, 10.5, forward=True)
  plt.show()
    
geographyPlot()